In [ ]:
#### IMPORTS ####

# Visualisation et manipulation des données
from collections import Counter
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

# Fichiers
import json
import os
import pickle
import zipfile

# Apprentissage
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE as tsne
from sklearn.neighbors import NearestNeighbors

# Interface
from ipywidgets import Button, Layout
from IPython.display import display

# Carte
from bokeh.io import curdoc
from bokeh.layouts import layout, column
from bokeh.models import ColumnDataSource, Div
from bokeh.models.tools import HoverTool
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.plotting import figure, show, output_file
import ipywidgets as widgets


In [ ]:
#### Chargement des fichiers 1/2 ####

# Fichier json d'apprentissage
with zipfile.ZipFile('../input/whats-cooking/train.json.zip', 'r') as z:
    with z.open('train.json') as f:
        datas = json.load(f) # id, cuisine, ingredients[]
    z.close()


In [ ]:
#### Chargement des fichiers 2/2 ####

# Fichier json dont on cherche les types de cuisines
with zipfile.ZipFile('../input/whats-cooking/test.json.zip', 'r') as z:
    with z.open('test.json') as f:
        datas2 = json.load(f) # id, ingredients[]
    z.close()


In [ ]:
#### Extraction des données des fichiers 1/2 ####

csv = []
max_ingr = 0
for data in datas:
    for ingredient in data['ingredients']:
        # Ajout de la ligne id-cuisine-ingredient
        tmp = dict()
        tmp['id'] = str(data['id'])
        tmp['cuisine'] = data['cuisine']
        tmp['ingredient'] = ingredient
        tmp['val'] = 1
        csv.append(tmp)
dfs = pd.DataFrame(csv) # DataFrame des données Source (d'apprentissage : train.json)


In [ ]:
#### Enregistrement des variables dans des fichiers avec Pickle 1/10 ####

# dfs
file = open('dfs.data', 'wb')
pickle.dump(dfs, file)
file.close()


In [ ]:
#### Extraction des données des fichiers 2/2 ####

csv2 = []
for data in datas:
    for ingredient in data['ingredients']:
        # Ajout de la ligne id-ingredient
        tmp = dict()
        tmp['id'] = str(data['id'])
        tmp['ingredient'] = ingredient
        tmp['val'] = 1
        csv2.append(tmp)

dft = pd.DataFrame(csv2) # DataFrame des données de Test (test.json)


In [ ]:
#### Récupération des données extraites 1/2 ####

cuisines = sorted(list(dfs['cuisine'].unique()))
ingredients = sorted(list(dfs['ingredient'].unique()))
id_plats_train = sorted(list(dfs['id'].unique()))


In [ ]:
#### Enregistrement des variables dans des fichiers avec Pickle 2/10 ####

# cuisines
file = open('cuisines.data', 'wb')
pickle.dump(cuisines, file)
file.close()

#### Enregistrement des variables dans des fichiers avec Pickle 3/10 ####

# ingredients
file = open('ingredients.data', 'wb')
pickle.dump(ingredients, file)
file.close()

#### Enregistrement des variables dans des fichiers avec Pickle 4/10 ####

# id_plats_train
file = open('id_plats_train.data', 'wb')
pickle.dump(id_plats_train, file)
file.close()


In [ ]:
#### Récupération des données extraites 2/2 ####

id_plats_test = sorted(list(dft['id'].unique()))


In [ ]:
#### Création des DataFrames d'apprentissage 1/2 ####

util_dfs = pd.pivot_table(data = dfs, values = 'val', index = 'id', columns = 'ingredient')
util_dfs = util_dfs.fillna(0)


In [ ]:
#### Création des DataFrames d'apprentissage 2/2 ####

util_dft = pd.pivot_table(data = dft, values = 'val', index = 'id', columns = 'ingredient')
util_dft = util_dft.fillna(0)


In [ ]:
#### Création et apprentissage du modèle de reconnaissance des types de cuisine ####

X = util_dfs

# Création du modèle
model = NMF(n_components = 6, init='random', random_state=0)

# Apprentissage du modèle (décomposition en deux matrices à 6 dimensions)
W = model.fit_transform(X)


In [ ]:
#### Enregistrement des variables dans des fichiers avec Pickle 5/10 ####

# model
file = open('model.data', 'wb')
pickle.dump(model, file)
file.close()


In [ ]:
#### Exploitation du modèle ####

X2 = util_dft # données de test

# Décomposition en deux matrices en utilisant le modèle
W2 = model.transform(X2)


In [ ]:
#### Création d'un modèle des voisins ####

neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(W)


In [ ]:
#### Enregistrement des variables dans des fichiers avec Pickle 6/10 ####

# neigh
file = open('neigh.data', 'wb')
pickle.dump(neigh, file)
file.close()


In [ ]:
#### Calcul des points en 2D correspondants aux plats dans l'ordre de W ####

points_plats = tsne(n_components = 2).fit_transform(W)


In [ ]:
#### Enregistrement des variables dans des fichiers avec Pickle 7/10 ####

# points_plats
file = open('points_plats.data', 'wb')
pickle.dump(points_plats, file)
file.close()


In [ ]:
#### Création des sets de données pour la carte 1/3 ####

points_par_pays = []
for pays in cuisines:
    ids = dfs.loc[dfs['cuisine'] == pays]['id'].unique()
    points = []
    for i in ids:
        index_plat = id_plats_train.index(i)
        coo = points_plats[index_plat]
        points.append([coo[0], coo[1]])
    points_par_pays.append(points)


In [ ]:
#### Enregistrement des variables dans des fichiers avec Pickle 8/10 ####

# points_par_pays
file = open('points_par_pays.data', 'wb')
pickle.dump(points_par_pays, file)
file.close()


In [ ]:
#### Création des sets de données pour la carte 2/3 ####

id_par_points_par_pays = []
for pays in cuisines:
    ids = dfs.loc[dfs['cuisine'] == pays]['id'].unique()
    points = []
    id_points = []
    for i in ids:
        id_points.append(i)
    id_par_points_par_pays.append(id_points)


In [ ]:
#### Enregistrement des variables dans des fichiers avec Pickle 9/10 ####

# id_par_points_par_pays
file = open('id_par_points_par_pays.data', 'wb')
pickle.dump(id_par_points_par_pays, file)
file.close()


In [ ]:
#### Création des sets de données pour la carte 3/3 ####

ingredients_par_points_par_pays = []
for pays in cuisines:
    ids = dfs.loc[dfs['cuisine'] == pays]['id'].unique()
    recettes_du_pays = []
    for i in ids:
        ingrs = ""
        for ingr in dfs.loc[dfs['id'] == i]['ingredient']:
            ingrs += ingr + ', '
        recettes_du_pays.append(ingrs)
    ingredients_par_points_par_pays.append(recettes_du_pays)


In [ ]:
#### Enregistrement des variables dans des fichiers avec Pickle 10/10 ####

# ingredients_par_points_par_pays
file = open('ingredients_par_points_par_pays.data', 'wb')
pickle.dump(ingredients_par_points_par_pays, file)
file.close()


In [ ]:
def dis_moi_qui_me_ressemble_le_plus(vecteur):
    voisins = neigh.kneighbors([vecteur], return_distance=False)[0]
    cuisines_voisines = []
    for voisin in voisins:
        cuisines_voisines.append(dfs.loc[dfs['id'] == id_plats_train[voisin]]['cuisine'].unique()[0])
    #print(cuisines_voisines)
    occurence_count = Counter(cuisines_voisines)
    return occurence_count.most_common(1)[0][0]


In [ ]:
nouvelle_recette = widgets.SelectMultiple(
    options=ingredients,
    value=['potatoes', 'raclette', 'ham'],
    description='Ingrédients :',
    disabled=False,
    layout = Layout(width='650px', height='200px'),
)
display(nouvelle_recette)


In [ ]:
print(nouvelle_recette.value)
nouvelle_recette_csv = []
for ingr in ingredients:
    tmp = dict()
    tmp['ingredient'] = ingr
    tmp['val'] = 1 if ingr in nouvelle_recette.value else 0
    tmp['id'] = 'Petit Poney'
    nouvelle_recette_csv.append(tmp)

nouvelle_recette_df = pd.DataFrame(nouvelle_recette_csv)
nouvelle_recette_util_df = pd.pivot_table(data = nouvelle_recette_df,
                                          values = 'val', index = 'id', columns = 'ingredient')
nouvelle_recette_W = model.transform(nouvelle_recette_util_df)[0]
print(nouvelle_recette_W)
print(dis_moi_qui_me_ressemble_le_plus(nouvelle_recette_W))


In [ ]:
del train, df, util_df, X
